# Data Preparation

We want to have a monthly episodic dataset so we can compute growth on hive attribute leading to more fair comparison than absolute value. Additonnally we will need the long format to validate any time dependence in the Cubee supplementation impact on survival and other attribute. 

The raw format has one line per inspection event so we want to normalize the pace to one row per month.

In [1]:
from datetime import datetime, date, timedelta

from scipy.stats import gmean
import pandas as pd 
import numpy as np 

from cubee.data.hive import Hive
from cubee.data.queen import Queen
from cubee.data.utils import simple_season
from cubee.configs.filepath import formatted_datapath,raw_datapath

In [2]:
AGE_LIMIT = 45
# Removing Nucs

### 0. Load data

In [3]:
d = pd.read_csv(raw_datapath)
print(len(d))
d.head()

39268


/var/folders/w6/lsz40xfn239dhtzy05m2r2jw0000gn/T/ipykernel_99565/1770239349.py:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  d = pd.read_csv(raw_datapath)


,tag_serial_number,hive_age,fob,fobr,foh,brood_pattern,grade,varroa,report_submitted_at,hive_identity_id,...,agressivity,operation_id,is_alive,queen_management,queen_status,state_province_long,country_long,city,yard_type,crop_type
0,277109,0,NaN,NaN,NaN,NaN,Weak,NaN,2022-03-17 19:53:17.868212+00:00,46954,...,NaN,159,1,NaN,NaN,Québec,Canada,Notre-Dame-de-la-Paix,outdoor wintering,Forest
1,277109,0,NaN,NaN,NaN,NaN,Weak,NaN,2022-03-17 19:53:50.348010+00:00,46954,...,NaN,159,1,NaN,NaN,Québec,Canada,Notre-Dame-de-la-Paix,outdoor wintering,Forest
2,277109,0,NaN,NaN,NaN,NaN,Weak,NaN,2022-03-17 19:53:50.348010+00:00,46954,...,NaN,159,1,NaN,NaN,Québec,Canada,Notre-Dame-de-la-Paix,outdoor wintering,Forest
3,277109,4,NaN,NaN,NaN,NaN,Medium,NaN,2022-03-21 20:08:43.433841+00:00,46954,...,NaN,159,1,NaN,NaN,Québec,Canada,Notre-Dame-de-la-Paix,outdoor wintering,Forest
4,277109,4,NaN,NaN,NaN,NaN,Medium,NaN,2022-03-21 20:08:43.433841+00:00,46954,...,NaN,159,1,NaN,NaN,Québec,Canada,Notre-Dame-de-la-Paix,outdoor wintering,Forest


In [4]:
d['report_submitted_at'] = pd.to_datetime(d['report_submitted_at'])
d['month'] = d['report_submitted_at'].dt.month
d['year'] = d['report_submitted_at'].dt.year 
d['date'] = d['report_submitted_at'].dt.date 

#### 0.1 Compute queen age 

In [5]:
exclude = ['queen cell present', 'queenless', 'capped queen cell',
           'introduced','double queen (for a later split)','queennotright']

queen_ages = []
queen_states = []

for hid in d.hive_identity_id.unique():
    one_hive = d.loc[d['hive_identity_id']==hid].reset_index(drop=True)
    queen_age = None
    queen_state = None
    born_date = None
    for _,row in one_hive.iterrows():
        qn, ts = row['action_detail'],row['report_submitted_at']
        if row['category'] == 'queen management':
            if qn not in exclude:
                if queen_age is None:
                    born_date = ts.date()
                    death_date = ts.date()
                    queen_age = 0
                    queen_state = False
                else:
                    death_date = ts.date()
                    queen_age = abs((death_date-born_date).days)
                    born_date = ts.date()
                    queen_state = True
        else:
            if born_date is not None:
                queen_age = abs((ts.date() - born_date).days)
                queen_state = False
            else:
                queen_age = None
                queen_state = None
        queen_ages.append(queen_age)
        queen_states.append(queen_state)
            
d['queen_age'] = queen_ages
d['queen_state'] = queen_states
d.head()

,tag_serial_number,hive_age,fob,fobr,foh,brood_pattern,grade,varroa,report_submitted_at,hive_identity_id,...,state_province_long,country_long,city,yard_type,crop_type,month,year,date,queen_age,queen_state
0,277109,0,NaN,NaN,NaN,NaN,Weak,NaN,2022-03-17 19:53:17.868212+00:00,46954,...,Québec,Canada,Notre-Dame-de-la-Paix,outdoor wintering,Forest,3,2022,2022-03-17,NaN,None
1,277109,0,NaN,NaN,NaN,NaN,Weak,NaN,2022-03-17 19:53:50.348010+00:00,46954,...,Québec,Canada,Notre-Dame-de-la-Paix,outdoor wintering,Forest,3,2022,2022-03-17,NaN,None
2,277109,0,NaN,NaN,NaN,NaN,Weak,NaN,2022-03-17 19:53:50.348010+00:00,46954,...,Québec,Canada,Notre-Dame-de-la-Paix,outdoor wintering,Forest,3,2022,2022-03-17,NaN,None
3,277109,4,NaN,NaN,NaN,NaN,Medium,NaN,2022-03-21 20:08:43.433841+00:00,46954,...,Québec,Canada,Notre-Dame-de-la-Paix,outdoor wintering,Forest,3,2022,2022-03-21,NaN,None
4,277109,4,NaN,NaN,NaN,NaN,Medium,NaN,2022-03-21 20:08:43.433841+00:00,46954,...,Québec,Canada,Notre-Dame-de-la-Paix,outdoor wintering,Forest,3,2022,2022-03-21,NaN,None


In [6]:
### Audio fake hive to be removed 
d = d.loc[d['tag_serial_number']!=277109].reset_index(drop=True)
d.head()

,tag_serial_number,hive_age,fob,fobr,foh,brood_pattern,grade,varroa,report_submitted_at,hive_identity_id,...,state_province_long,country_long,city,yard_type,crop_type,month,year,date,queen_age,queen_state
0,277108,0,NaN,NaN,NaN,NaN,Medium,NaN,2022-03-17 20:02:16.658582+00:00,46955,...,Québec,Canada,Notre-Dame-de-la-Paix,outdoor wintering,Forest,3,2022,2022-03-17,NaN,None
1,277108,0,NaN,NaN,NaN,NaN,Medium,NaN,2022-03-17 20:03:17.864504+00:00,46955,...,Québec,Canada,Notre-Dame-de-la-Paix,outdoor wintering,Forest,3,2022,2022-03-17,NaN,None
2,277108,0,NaN,NaN,NaN,NaN,Medium,NaN,2022-03-17 20:03:17.864504+00:00,46955,...,Québec,Canada,Notre-Dame-de-la-Paix,outdoor wintering,Forest,3,2022,2022-03-17,NaN,None
3,277108,4,NaN,NaN,NaN,NaN,Medium,NaN,2022-03-21 20:08:43.433841+00:00,46955,...,Québec,Canada,Notre-Dame-de-la-Paix,outdoor wintering,Forest,3,2022,2022-03-21,NaN,None
4,277108,4,NaN,NaN,NaN,NaN,Medium,NaN,2022-03-21 20:08:43.433841+00:00,46955,...,Québec,Canada,Notre-Dame-de-la-Paix,outdoor wintering,Forest,3,2022,2022-03-21,NaN,None


In [7]:
d.columns

Index(['tag_serial_number', 'hive_age', 'fob', 'fobr', 'foh', 'brood_pattern',
       'grade', 'varroa', 'report_submitted_at', 'hive_identity_id',
       'category', 'action_detail', 'report_report_type', 'yard_name',
       'report_notes', 'lon', 'lat', 'agressivity', 'operation_id', 'is_alive',
       'queen_management', 'queen_status', 'state_province_long',
       'country_long', 'city', 'yard_type', 'crop_type', 'month', 'year',
       'date', 'queen_age', 'queen_state'],
      dtype='object')

***

### 1.Build dataset

Here we fetch value based on the data collection process.

- We use monthly value for each hive attribute

***

#### 1.1 Hive level

In [8]:
def determine_age_and_state(x):
    if x['queen_state'].max():
        queen_age = x['queen_age'].max()
    else:
        queen_age = x['queen_age'].values[-1]
    return x['is_alive'].min(), x['hive_age'].max(), queen_age, x['queen_state'].max()

In [9]:
def determine_cubee_supplementation_event(x):
    c = 0 
    for _,row in x.iterrows():
        if (row['action_detail'] == 'mushroom') | (row['category'] == 'cubee ml'):
            c=+ 1
    return c 

In [10]:
def determine_fob_fobr_varroa_foh(x):
    v = x['varroa'].min()
    if x['fob'].isnull().mean() < 1.0:
        fob= gmean(x['fob'].fillna(x['fob'].mean()))
    else:
        fob= x['fob'].mean()
    if x['fobr'].isnull().mean() < 1.0:
        fobr= gmean(x['fobr'].fillna(x['fobr'].mean()))
    else:
        fobr= x['fobr'].mean()

    if x['foh'].isnull().mean() < 1.0:
        foh= gmean(x['foh'].fillna(x['foh'].mean()))
    else:
        foh= x['foh'].mean()
    return v, fob, fobr, foh

In [11]:
def determine_brood_pattern(x):
    if x['brood_pattern'].isnull().mean() < 1.0:
        return gmean(x['brood_pattern'].fillna(x['brood_pattern'].mean()))
    else:
        return x['brood_pattern'].mean()

In [12]:
def determine_stings(x):
    if x['agressivity'].isnull().mean() < 1.0:
        return gmean(x['agressivity'].fillna(x['agressivity'].mean()))
    else:
        return x['agressivity'].mean()

In [13]:
len(d['hive_identity_id'].unique())

326

In [14]:
def determine_interactions(full, yid, start_of_month, end_of_month):
    subset_full = full.loc[(full['date'] >= start_of_month.date()) & (full['date'] < end_of_month) & (full['yard_name'] ==yid)].reset_index(drop=True)
    return len(subset_full['hive_identity_id'].unique()) - 1

In [15]:
final = []
for hid in d.hive_identity_id.unique():
    queen_id = 0
    one_hive = d.loc[d['hive_identity_id']==hid].reset_index(drop=True)
    one_hive = one_hive.sort_values("report_submitted_at",ascending=True).reset_index(drop=True)
    first_obs = one_hive['report_submitted_at'].min().date() - pd.offsets.MonthBegin()
    last_obs = date.today() + pd.offsets.MonthEnd() if one_hive['is_alive'].min() == 1 else one_hive['date'].max() + pd.offsets.MonthEnd()
    dates = [x.date() for x in pd.date_range(start=first_obs, end=last_obs, freq='ME',inclusive="both").tolist() ]
    results = {d:{"time1":None, "time2":None,"state":None, "hive_age":None, "cubee":0, "fob":None,"fobr":None,"foh":None,"varroa":None,"queen_age":None,"queen_state":None,"queen_cubee":None,"queen_id":None, "brood_pattern":None,"stings":None,"hid":hid,"hive_interacted":None} for d in dates}
    for i,end_of_month in enumerate(dates):
        start_of_month = end_of_month - pd.offsets.MonthBegin()
        days_in_month = (end_of_month - start_of_month.date()).days
        subset = one_hive.loc[(one_hive['date'] >= start_of_month.date()) & (one_hive['date'] < end_of_month)].reset_index(drop=True)
        if len(subset) > 0:
            results[end_of_month]["state"], results[end_of_month]["hive_age"], results[end_of_month]['queen_age'],results[end_of_month]['queen_state'] = determine_age_and_state(subset)
            results[end_of_month]["cubee"] = determine_cubee_supplementation_event(subset)
            results[end_of_month]['varroa'], results[end_of_month]['fob'], results[end_of_month]['fobr'],results[end_of_month]['foh'] = determine_fob_fobr_varroa_foh(subset)
            results[end_of_month]['brood_pattern'] = determine_brood_pattern(subset)
            results[end_of_month]['stings'] = determine_stings(subset)
            results[end_of_month]['hive_interacted'] = determine_interactions(d, subset['yard_name'].values[-1], start_of_month, end_of_month)
            if results[end_of_month]['queen_age'] > 0:
                results[end_of_month]['queen_id'] = f"{hid}_queen_{queen_id}"
                if results[end_of_month]['queen_state']:
                    queen_id += 1
            if i == 0:
                results[end_of_month]['time1'] = 0 
                results[end_of_month]['time2'] = days_in_month 
            else:
                if results[dates[i-1]]['time2']  is not None:
                    results[end_of_month]['time1'] = results[dates[i-1]]['time2'] 
                else:
                    results[end_of_month]['time1'] = 0 
                results[end_of_month]['time2'] = results[end_of_month]['time1'] + days_in_month 
            final.append(results[end_of_month])

    

In [16]:
final_df = pd.DataFrame(final)
final_df.tail()

,time1,time2,state,hive_age,cubee,fob,fobr,foh,varroa,queen_age,queen_state,queen_cubee,queen_id,brood_pattern,stings,hid,hive_interacted
2844,29,59,1,32,1,3.000000,6.0,NaN,5.0,32.0,False,None,664202_queen_0,5.0,0.0,664202,31
2845,0,29,1,0,0,2.000000,2.0,NaN,NaN,0.0,False,None,None,NaN,NaN,665047,34
2846,29,59,1,47,0,4.162766,2.0,NaN,0.0,47.0,False,None,665047_queen_0,NaN,NaN,665047,31
2847,0,29,1,3,0,5.000000,3.0,2.0,NaN,3.0,False,None,665312_queen_0,NaN,0.0,665312,9
2848,29,59,1,38,0,6.000000,3.0,2.0,5.0,38.0,False,None,665312_queen_0,NaN,0.0,665312,9


In [17]:
final_df.loc[final_df['hid']==661318]

,time1,time2,state,hive_age,cubee,fob,fobr,foh,varroa,queen_age,queen_state,queen_cubee,queen_id,brood_pattern,stings,hid,hive_interacted
2839,0,29,1,7,1,7.639437,5.0,3.0,NaN,7.0,False,None,661318_queen_0,NaN,0.0,661318,7
2840,29,59,1,39,0,9.000000,5.0,3.0,30.0,39.0,False,None,661318_queen_0,NaN,0.0,661318,7


In [18]:
final_df.describe()

,time1,time2,state,hive_age,cubee,fob,fobr,foh,varroa,queen_age,brood_pattern,stings,hid,hive_interacted
count,2849.000000,2849.000000,2849.000000,2849.000000,2849.000000,2346.000000,2727.000000,2319.000000,683.000000,1779.000000,1866.000000,1952.000000,2849.000000,2849.000000
mean,67.521587,97.020007,0.918217,222.048789,0.179010,10.123490,4.469735,5.081475,6.102489,139.730748,4.114590,1.336204,206183.229203,17.971920
std,61.497433,61.615625,0.274082,221.616527,0.383428,6.397649,2.684053,4.215452,11.928233,147.201992,1.055158,1.973044,167647.461519,10.841811
min,0.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,46955.000000,0.000000
25%,0.000000,30.000000,1.000000,62.000000,0.000000,5.000000,2.423492,2.000000,0.000000,36.000000,4.000000,0.000000,68015.000000,7.000000
50%,59.000000,89.000000,1.000000,135.000000,0.000000,8.179992,4.000000,4.000000,1.000000,84.000000,4.298446,0.000000,157146.000000,21.000000
75%,118.000000,148.000000,1.000000,327.000000,0.000000,13.997480,6.000000,6.539178,5.000000,198.500000,5.000000,2.000000,314954.000000,24.000000
max,381.000000,411.000000,1.000000,1239.000000,1.000000,35.000000,20.000000,32.166238,91.000000,802.000000,5.000000,10.000000,665312.000000,42.000000


### Exclude very young hives

In [19]:
excluded_hives = final_df.groupby("hid").max(numeric_only=True).loc[final_df.groupby("hid").max(numeric_only=True)['hive_age'] < AGE_LIMIT].index

In [20]:
final_df = final_df.loc[~final_df['hid'].isin(excluded_hives)]

In [21]:
final_df.to_csv(formatted_datapath,index=False)